In [7]:
import importlib
import sys
import torch
from torch.utils.data import random_split
import random
from collections.abc import Iterator
from tqdm.notebook import tqdm
from Levenshtein import distance
import collections
import pickle
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import CRPS.CRPS as pscore
import numpy as np
import os
import pickle
from multiprocessing import Pool, Manager


sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')

from model.dropout_uncertainty_enc_dec_LSTM_new_dec.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [ ]:
processed_prefixes = set()


def process_file(file_path):
    """
    Process a single pickle file and return its status along with any keys if processed.
    """
    try:
        with open(file_path, "rb") as f:
            data = pickle.load(f)
        if not isinstance(data, dict):
            print("No dict:", file_path)
            return (file_path, "Skipping: Not a dictionary", None)
        else:
            print("Processed:", file_path)
            return (file_path, "Processed", list(data.keys()))
    except Exception as e:
        print("Error:", file_path)
        return (file_path, f"Error: {e}", None)
    

input_dir  = '../../../../evaluation_results_ssd/BPIC17/gn'

files = [os.path.join(input_dir, fname) 
            for fname in os.listdir(input_dir) if fname.endswith(".pkl")]


# Create a pool of workers
with Pool(processes=2) as pool:
    results = pool.map(process_file, files)

processed_prefixes = set()
# Process the results in the main process
for file_path, status, keys in results:
    print(f"{status} - {os.path.basename(file_path)}")
    if keys is not None:
        # Extend the shared list with keys from this file
        processed_prefixes.update(keys)

# Optionally, convert to a set to remove duplicates
print(f"Unique processed prefixes: {processed_prefixes}")


with open('BPIC2017_processed_prefixes.pkl', 'wb') as f:
    pickle.dump(processed_prefixes, f)




Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_009450.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_036300.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_014050.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_024350.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_053600.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_018650.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_052200.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_043350.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_041550.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_012300.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_028050.pkl
Processed: ../../../../evaluation_results_ssd/BPIC17/gn/results_part_019750.pkl
Processed: ../../../../evaluation_result

BrokenPipeError: [Errno 32] Broken pipe

In [23]:
processed_prefixes

{('Application_1373369508', 9),
 ('Application_1104878281', 13),
 ('Application_1552182273', 58),
 ('Application_1186519754', 34),
 ('Application_1110160976', 55),
 ('Application_1477137664', 16),
 ('Application_1433157049', 8),
 ('Application_1224343980', 9),
 ('Application_1210979709', 17),
 ('Application_1403322284', 13),
 ('Application_1027284246', 56),
 ('Application_1480288163', 6),
 ('Application_1377765652', 36),
 ('Application_1367761225', 31),
 ('Application_1362016551', 20),
 ('Application_156087508', 48),
 ('Application_1036785189', 18),
 ('Application_1143890425', 29),
 ('Application_1493375085', 6),
 ('Application_1111168705', 4),
 ('Application_1387679604', 25),
 ('Application_1272466781', 27),
 ('Application_1306392109', 38),
 ('Application_1197309763', 35),
 ('Application_1450091917', 18),
 ('Application_1309054175', 78),
 ('Application_1317579416', 14),
 ('Application_1383530613', 35),
 ('Application_1312126160', 6),
 ('Application_1160839680', 7),
 ('Application_1485

In [ ]:
# Load the dataset
file_path_data_set = '../../../encoded_data/BPIC_2017_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

import src.evaluation.probabilistic_evaluation
importlib.reload(src.evaluation.probabilistic_evaluation)
from src.evaluation.probabilistic_evaluation import ProbabilisticEvaluation

#device = torch.device("cuda")  # Use the first available GPU
#model = model.to(device)

new_eval = ProbabilisticEvaluation(None, bpic_17_test_dataset,
                                   num_processes=32,
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   )

all_prefixes = set()
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.count_only(random_order=False)):
    all_prefixes.add((case_name, prefix_len))


remaining_prefixes = all_prefixes - processed_prefixes

with open('BPIC2017_remaining_prefixes.pkl', 'wb') as f:
    pickle.dump(remaining_prefixes, f)

print(len(all_prefixes))
print(len(remaining_prefixes))

print(remaining_prefixes)


Exception ignored in: <_io.BytesIO object at 0x7fb18e04b7e0>
Traceback (most recent call last):
  File "/home/LordKunkler/PrimePPM/src/notebooks/evaluation_run_notebooks/../../event_log_loader/new_event_log_loader.py", line 362, in __getitem__
    cat.append(i[idx])
BufferError: Existing exports of data: object cannot be re-sized


  0%|          | 0/4682 [00:00<?, ?it/s]

TypeError: unhashable type: 'set'